In [26]:
import os 
import shutil
import random
import glob

In [ ]:
# !mkdir spider_test
# !mkdir spider_test/normal
# !mkdir spider_test/abnormal

In [ ]:
# # Set random seed for reproducible results
# random.seed(42)

# # Collect all image files first
# normal_files = []
# abnormal_files = []

# for root, dirs, files in os.walk("/Users/zideng/Work/LDAWT/examples/black_widows_6_17"):
#     for file in files:
#         if file.lower().endswith((".jpg", ".png", ".jpeg")):
#             full_path = os.path.join(root, file)
#             if "Latrodectus_hesperus" in root:
#                 abnormal_files.append(full_path)
#             else:
#                 normal_files.append(full_path)

# # Randomly sample the desired number of files
# normal_sample = random.sample(normal_files, min(100, len(normal_files)))
# abnormal_sample = random.sample(abnormal_files, min(200, len(abnormal_files)))

# # Copy the sampled files
# for file_path in normal_sample:
#     filename = os.path.basename(file_path)
#     shutil.copy(file_path, os.path.join("spider_test/normal", filename))

# for file_path in abnormal_sample:
#     filename = os.path.basename(file_path)
#     shutil.copy(file_path, os.path.join("spider_test/abnormal", filename))

# print(f"Copied {len(normal_sample)} files to normal folder")
# print(f"Copied {len(abnormal_sample)} files to abnormal folder")
    

Copied 100 files to normal folder
Copied 200 files to abnormal folder


In [27]:
# 1. Import required modules
from anomalib.data import MVTecAD
from anomalib.deploy import ExportType
from anomalib.engine import Engine
from anomalib.models import Patchcore
from anomalib.data import Folder
import albumentations as A
from albumentations.pytorch import ToTensorV2
import numpy as np
import torch
from PIL import Image
import matplotlib.pyplot as plt

In [28]:
def albu_adapter(aug):
    """Wrap an Albumentations Compose so it accepts (image) and returns a Tensor.
       Converts PIL/Tensor to NumPy HxWxC uint8 before calling Albumentations.
    """
    def _call(image):
        # 1) Normalize input type to NumPy HxWxC
        if isinstance(image, np.ndarray):
            img = image
            # If CHW, convert to HWC
            if img.ndim == 3 and img.shape[0] in (1, 3, 4) and img.shape[2] not in (1, 3, 4):
                img = np.transpose(img, (1, 2, 0))
        elif isinstance(image, Image.Image):
            img = np.array(image)  # PIL -> HWC uint8
        elif isinstance(image, torch.Tensor):
            arr = image.detach().cpu().numpy()
            # Assume CHW; convert to HWC
            if arr.ndim == 3 and arr.shape[0] in (1, 3, 4):
                arr = np.transpose(arr, (1, 2, 0))
            # If float in [0,1], convert to uint8 0..255 for Albumentations Normalize defaults
            if arr.dtype != np.uint8:
                arr = np.clip(arr, 0, 1) * 255.0
                arr = arr.astype(np.uint8)
            img = arr
        else:
            raise TypeError(f"Unsupported image type: {type(image)}. Expected numpy, PIL, or torch.Tensor.")

        # Ensure 3rd dimension exists for grayscale
        if img.ndim == 2:
            img = img[..., None]

        # 2) Call Albumentations with named argument
        out = aug(image=img)
        # out["image"] is a torch.Tensor thanks to ToTensorV2()
        return out["image"]
    return _call

# --- Define transforms (Albumentations still) ---
train_aug = A.Compose([
    A.Resize(256, 256),
    # Optional robustifying augs for field images:
    # A.HorizontalFlip(p=0.5),
    # A.ColorJitter(p=0.2),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),  # expects uint8 input
    ToTensorV2(),
])

eval_aug = A.Compose([
    A.Resize(256, 256),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

train_tf = albu_adapter(train_aug)
eval_tf  = albu_adapter(eval_aug)


In [29]:
dm = Folder(
    name="spider_anomaly_detection",
    root="spider_test",
    normal_dir="normal",
    abnormal_dir="abnormal",
    train_augmentations=train_tf,
    val_augmentations=eval_tf,
    test_augmentations=eval_tf,
    train_batch_size=16,
    eval_batch_size=16,
    num_workers=0,  # set to 0 for easier debugging; increase later
)

In [46]:
# FIXED WEIGHTS & BIASES INTEGRATION FOR SPIDERANOMALY TEST
# Install wandb: pip install wandb
# Login first time: wandb login

import wandb
from anomalib.loggers import AnomalibWandbLogger

# Initialize wandb logger (AnomalibWandbLogger for anomalib integration)
wandb_logger = AnomalibWandbLogger(
    project="spider-anomaly-detection-test2",
    name="patchcore-spider-experiment",
    save_dir="./logs"  # Required parameter for AnomalibWandbLogger
)

# Initialize wandb run with config (for manual logging)
wandb.init(
    project="spider-anomaly-detection-test2",
    name="patchcore-spider-experiment",
    tags=["patchcore", "spiders", "anomaly-detection", "resnet18"],
    notes="Spider anomaly detection using PatchCore with detailed evaluation",
    config={
        "model": "PatchCore",
        "backbone": "resnet18", 
        "layers": ["layer2", "layer3"],
        "coreset_sampling_ratio": 0.10,
        "num_neighbors": 12,
        "train_batch_size": 16,
        "eval_batch_size": 16,
        "max_epochs": 1,
        "dataset_root": "spider_test",
        "image_size": "default",
        "num_workers": 0,
    }
)

print("🚀 Weights & Biases logger initialized!")
print(f"📊 Project: {wandb.run.project}")
print(f"🏷️  Run name: {wandb.run.name}")
print(f"🌐 Run URL: {wandb.run.url}")


🚀 Weights & Biases logger initialized!
📊 Project: spider-anomaly-detection-test2
🏷️  Run name: patchcore-spider-experiment
🌐 Run URL: https://wandb.ai/zkdeng-university-of-arizona/spider-anomaly-detection-test2/runs/llf9tzqr


In [50]:
# TRAINING WITH WANDB LOGGING INTEGRATION - FIXED VERSION
import warnings
import time
warnings.filterwarnings("ignore", category=UserWarning, module="torchmetrics")

start_time = time.time()

# Create model (same as before)
model = Patchcore(
    backbone="resnet18",
    layers=["layer2", "layer3"],
    pre_trained=True,
    coreset_sampling_ratio=0.10,
    num_neighbors=12,
    visualizer=False,
)

# Create engine WITH wandb logger integration
engine = Engine(
    max_epochs=1, 
    accelerator="auto", 
    devices=1,
    logger=wandb_logger,  # 🔥 Add wandb logger here!
    enable_progress_bar=True,
    log_every_n_steps=1,  # Log metrics every step
    enable_checkpointing=True,  # Save model checkpoints to wandb
)

print("🚀 Starting training with wandb logging...")
engine.fit(datamodule=dm, model=model)

print("\n🧪 Running evaluation with wandb logging...")
test_results = engine.test(datamodule=dm, model=model)

post_processor = model.post_processor

# Check different possible threshold attributes
threshold_value = None
threshold_attrs = ['threshold', 'threshold_', 'image_threshold', 'pixel_threshold']

for attr in threshold_attrs:
    if hasattr(post_processor, attr):
        threshold_value = getattr(post_processor, attr)
        #print(f"Optimal Decision Threshold: {threshold_value}")
        #print(f"Threshold Attribute: {attr}")
        break

if threshold_value is None:
    print("🔍 Available post_processor attributes:")
    print([attr for attr in dir(post_processor) if not attr.startswith('_')])
    print("Threshold not found in standard attributes")

#print(f"📊 Threshold Method: {type(post_processor).__name__}")

#print("\n📈 DETAILED PERFORMANCE METRICS:")
#print("=" * 50)
# for metric, value in test_results[0].items():
#     print(f"{metric:25s}: {value:.6f}")

#print(f"\n🎯 OPTIMAL THRESHOLD DETAILS:")
#print("=" * 50)
if threshold_value is not None:
    print(f"Threshold Value       : {threshold_value:.6f}")
else:
    print(f"Threshold Value       : Not accessible")
print(f"Threshold Type        : {type(post_processor).__name__}")

if hasattr(post_processor, 'normalization_method'):
    print(f"Normalization Method  : {post_processor.normalization_method}")
    
print(f"\n💡 MODEL CONFIGURATION:")
print("=" * 50)

# Access model configuration safely through hparams or config
if hasattr(model, 'hparams'):
    config = model.hparams
    display(config)
else:
    # Fallback: show available attributes
    print("Model attributes (first 10):")
    attrs = [attr for attr in dir(model) if not attr.startswith('_')][:10]
    print(attrs)
    print("Standard configuration attributes not accessible")
print(f"Time taken           : {time.time() - start_time:.2f} seconds")


# Log basic test results to wandb
basic_metrics = {
    "test/image_AUROC": test_results[0].get("image_AUROC", 0),
    "test/image_F1Score": test_results[0].get("image_F1Score", 0),
    "test/threshold_value": threshold_value,
    "test/threshold_type": type(post_processor).__name__,
    "test/model_configuration": config,
    "training/duration_seconds": time.time() - start_time,
}

# Use wandb directly for logging instead of wandb_logger.experiment
wandb.log(basic_metrics)

print(f"\n📈 BASIC TEST RESULTS:")
for metric, value in test_results[0].items():
    print(f"{metric:20s}: {value:.6f}")

print(f"\n⏱️  Training completed in: {time.time() - start_time:.2f} seconds")
print(f"🌐 View results at: {wandb.run.url}")


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/zideng/mamba/envs/ml/lib/python3.13/site-packages/lightning/pytorch/core/optimizer.py:183: `LightningModule.configure_optimizers` returned `None`, this fit will run with no optimizer

  | Name           | Type           | Params | Mode 
----------------------------------------------------------
0 | pre_processor  | PreProcessor   | 0      | train
1 | post_processor | PostProcessor  | 0      | train
2 | evaluator      | Evaluator      | 0      | train
3 | model          | PatchcoreModel | 2.8 M  | train
----------------------------------------------------------
2.8 M     Trainable params
0         Non-trainable params
2.8 M     Total params
11.131    Total estimated model params size (MB)
19        Modules in train mode
69        Modules in eval mode
/Users/zideng/mamba/envs/ml/lib/python3.13/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: T

🚀 Starting training with wandb logging...
Epoch 0: 100%|██████████| 5/5 [00:00<00:00,  8.07it/s, v_num=tzqr]



























































































































































Selecting Coreset Indices.: 100%|██████████| 6272/6272 [00:16<00:00, 386.89it/s]


Epoch 0: 100%|██████████| 5/5 [00:17<00:00,  0.28it/s, v_num=tzqr]

`Trainer.fit` stopped: `max_epochs=1` reached.


...


🧪 Running evaluation with wandb logging...
Testing DataLoader 0: 100%|██████████| 7/7 [00:00<00:00,  8.23it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        image_AUROC        │     0.628000020980835     │
│       image_F1Score       │    0.9523809552192688     │
└───────────────────────────┴───────────────────────────┘

Threshold Value       : 23.215326
Threshold Type        : PostProcessor

💡 MODEL CONFIGURATION:


"backbone":               resnet18
"coreset_sampling_ratio": 0.1
"evaluator":              True
"layers":                 ['layer2', 'layer3']
"num_neighbors":          12
"post_processor":         True
"pre_processor":          True
"pre_trained":            True
"visualizer":             False

Time taken           : 19.36 seconds

📈 BASIC TEST RESULTS:
image_AUROC         : 0.628000
image_F1Score       : 0.952381

⏱️  Training completed in: 19.36 seconds
🌐 View results at: https://wandb.ai/zkdeng-university-of-arizona/spider-anomaly-detection-test2/runs/llf9tzqr


In [ ]:
# TRAINING WITH WANDB LOGGING INTEGRATION
import warnings
import time
warnings.filterwarnings("ignore", category=UserWarning, module="torchmetrics")

start_time = time.time()

# Create model (same as before)
model = Patchcore(
    backbone="resnet18",
    layers=["layer2", "layer3"],
    pre_trained=True,
    coreset_sampling_ratio=0.10,
    num_neighbors=12,
    visualizer=False,
)

# Create engine WITH wandb logger integration
engine = Engine(
    max_epochs=1, 
    accelerator="auto", 
    devices=1,
    logger=wandb_logger,  # 🔥 Add wandb logger here!
    enable_progress_bar=True,
    log_every_n_steps=1,  # Log metrics every step
    enable_checkpointing=True,  # Save model checkpoints to wandb
)

print("🚀 Starting training with wandb logging...")
engine.fit(datamodule=dm, model=model)

print("\n🧪 Running evaluation with wandb logging...")
test_results = engine.test(datamodule=dm, model=model)

# Log basic test results to wandb
basic_metrics = {
    "test/image_AUROC": test_results[0].get("image_AUROC", 0),
    "test/image_F1Score": test_results[0].get("image_F1Score", 0),
    "training/duration_seconds": time.time() - start_time,
}

wandb_logger.experiment.log(basic_metrics)

print(f"\n📈 BASIC TEST RESULTS:")
for metric, value in test_results[0].items():
    print(f"{metric:20s}: {value:.6f}")

print(f"\n⏱️  Training completed in: {time.time() - start_time:.2f} seconds")
print(f"🌐 View results at: {wandb_logger.experiment.url}")


In [44]:
# ADVANCED WANDB LOGGING WITH DETAILED ANALYSIS
import numpy as np

def log_detailed_analysis_to_wandb(analysis_results, wandb_logger):
    """Log comprehensive analysis results to wandb."""
    
    if analysis_results is None:
        print("⚠️ No analysis results to log")
        return
    
    print("📊 Logging detailed analysis to wandb...")
    
    # Extract key metrics
    metrics = analysis_results['metrics']
    cm = analysis_results['confusion_matrix']
    
    # Log detailed performance metrics
    detailed_metrics = {
        "detailed/auroc": metrics['auroc'],
        "detailed/accuracy": metrics['accuracy'],
        "detailed/precision": metrics['precision'],
        "detailed/recall": metrics['recall'],
        "detailed/specificity": metrics['specificity'],
        "detailed/f1_score": metrics['f1_score'],
        
        "confusion_matrix/true_positives": metrics['tp'],
        "confusion_matrix/true_negatives": metrics['tn'],
        "confusion_matrix/false_positives": metrics['fp'],
        "confusion_matrix/false_negatives": metrics['fn'],
        
        "threshold/optimal_value": analysis_results['threshold'] if analysis_results['threshold'] is not None else 0,
        
        "scores/min_anomaly_score": np.min(analysis_results['anomaly_scores']) if len(analysis_results['anomaly_scores']) > 0 else 0,
        "scores/max_anomaly_score": np.max(analysis_results['anomaly_scores']) if len(analysis_results['anomaly_scores']) > 0 else 0,
        "scores/mean_score": np.mean(analysis_results['anomaly_scores']) if len(analysis_results['anomaly_scores']) > 0 else 0,
    }
    
    # Add score distributions by class
    if len(analysis_results['anomaly_scores']) > 0 and len(analysis_results['ground_truths']) > 0:
        normal_mask = analysis_results['ground_truths'] == 0
        abnormal_mask = analysis_results['ground_truths'] == 1
        
        if np.any(normal_mask):
            detailed_metrics["scores/mean_normal_score"] = np.mean(analysis_results['anomaly_scores'][normal_mask])
        if np.any(abnormal_mask):
            detailed_metrics["scores/mean_abnormal_score"] = np.mean(analysis_results['anomaly_scores'][abnormal_mask])
    
    # Log all metrics to wandb
    wandb_logger.experiment.log(detailed_metrics)
    
    # Create confusion matrix table for wandb
    if cm.size > 0:
        cm_table = wandb.Table(
            columns=["Actual", "Predicted", "Count"],
            data=[
                ["Normal", "Normal", int(cm[0,0])],
                ["Normal", "Abnormal", int(cm[0,1])],
                ["Abnormal", "Normal", int(cm[1,0])],
                ["Abnormal", "Abnormal", int(cm[1,1])]
            ]
        )
        wandb_logger.experiment.log({"confusion_matrix_table": cm_table})
    
    # Log score distribution histogram
    if len(analysis_results['anomaly_scores']) > 0:
        wandb_logger.experiment.log({
            "score_distribution": wandb.Histogram(analysis_results['anomaly_scores'])
        })
    
    print("✅ Detailed analysis logged to wandb!")
    return detailed_metrics

# Check if we have analysis results from previous evaluation
if 'analysis_results' in locals() and analysis_results is not None:
    print("📊 Found existing analysis results, logging to wandb...")
    logged_metrics = log_detailed_analysis_to_wandb(analysis_results, wandb_logger)
else:
    print("🔄 Running detailed analysis for wandb logging...")
    # Run the detailed analysis if it hasn't been done yet
    analysis_results = get_proper_analysis(engine, model, dm)
    if analysis_results is not None:
        logged_metrics = log_detailed_analysis_to_wandb(analysis_results, wandb_logger)
    else:
        print("❌ Could not generate analysis results for wandb logging")


ckpt_path is not provided. Model weights will not be loaded.
The following callbacks returned in `LightningModule.configure_callbacks` will override existing callbacks passed to Trainer: Evaluator, ImageVisualizer, PostProcessor, PreProcessor


🔄 Running detailed analysis for wandb logging...
🔬 PERFORMING PROPER ANOMALIB EVALUATION...
Processing 7 batches...
Batch 1/7
Batch type: <class 'anomalib.data.dataclasses.torch.image.ImageBatch'>
Batch attributes: ['image', 'gt_label', 'gt_mask', 'mask_path', 'anomaly_map', 'pred_score', 'pred_mask', 'pred_label', 'explanation', 'image_path']
  ✅ Found labels in batch.gt_label
  Image tensor shape: torch.Size([16, 3, 256, 256])


/Users/zideng/mamba/envs/ml/lib/python3.13/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.


Predicting DataLoader 0:  14%|█▍        | 1/7 [00:00<00:05,  1.03it/s]

Field 'gt_mask' is None in ImageItem. Skipping visualization.
Field 'gt_mask' is None in ImageItem. Skipping visualization.
Field 'gt_mask' is None in ImageItem. Skipping visualization.
Field 'gt_mask' is None in ImageItem. Skipping visualization.
Field 'gt_mask' is None in ImageItem. Skipping visualization.
Field 'gt_mask' is None in ImageItem. Skipping visualization.
Field 'gt_mask' is None in ImageItem. Skipping visualization.
Field 'gt_mask' is None in ImageItem. Skipping visualization.
Field 'gt_mask' is None in ImageItem. Skipping visualization.
Field 'gt_mask' is None in ImageItem. Skipping visualization.
Field 'gt_mask' is None in ImageItem. Skipping visualization.
Field 'gt_mask' is None in ImageItem. Skipping visualization.
Field 'gt_mask' is None in ImageItem. Skipping visualization.
Field 'gt_mask' is None in ImageItem. Skipping visualization.
Field 'gt_mask' is None in ImageItem. Skipping visualization.
Field 'gt_mask' is None in ImageItem. Skipping visualization.


...

Predicting DataLoader 0:  86%|████████▌ | 6/7 [00:01<00:00,  3.44it/s]

Field 'gt_mask' is None in ImageItem. Skipping visualization.
Field 'gt_mask' is None in ImageItem. Skipping visualization.
Field 'gt_mask' is None in ImageItem. Skipping visualization.
Field 'gt_mask' is None in ImageItem. Skipping visualization.
Field 'gt_mask' is None in ImageItem. Skipping visualization.
Field 'gt_mask' is None in ImageItem. Skipping visualization.
Field 'gt_mask' is None in ImageItem. Skipping visualization.
Field 'gt_mask' is None in ImageItem. Skipping visualization.
Field 'gt_mask' is None in ImageItem. Skipping visualization.
Field 'gt_mask' is None in ImageItem. Skipping visualization.
Field 'gt_mask' is None in ImageItem. Skipping visualization.
Field 'gt_mask' is None in ImageItem. Skipping visualization.
Field 'gt_mask' is None in ImageItem. Skipping visualization.
Field 'gt_mask' is None in ImageItem. Skipping visualization.
Field 'gt_mask' is None in ImageItem. Skipping visualization.
Field 'gt_mask' is None in ImageItem. Skipping visualization.


Predicting DataLoader 0: 100%|██████████| 7/7 [00:02<00:00,  3.35it/s]

Field 'gt_mask' is None in ImageItem. Skipping visualization.
Field 'gt_mask' is None in ImageItem. Skipping visualization.
Field 'gt_mask' is None in ImageItem. Skipping visualization.
Field 'gt_mask' is None in ImageItem. Skipping visualization.
Field 'gt_mask' is None in ImageItem. Skipping visualization.
Field 'gt_mask' is None in ImageItem. Skipping visualization.
Field 'gt_mask' is None in ImageItem. Skipping visualization.
Field 'gt_mask' is None in ImageItem. Skipping visualization.
Field 'gt_mask' is None in ImageItem. Skipping visualization.
Field 'gt_mask' is None in ImageItem. Skipping visualization.
Field 'gt_mask' is None in ImageItem. Skipping visualization.
Field 'gt_mask' is None in ImageItem. Skipping visualization.
Field 'gt_mask' is None in ImageItem. Skipping visualization.
Field 'gt_mask' is None in ImageItem. Skipping visualization.


Predicting DataLoader 0: 100%|██████████| 7/7 [00:02<00:00,  2.93it/s]
Got 7 prediction results
Prediction results type: <class 'list'>
First prediction type: <class 'anomalib.data.dataclasses.torch.image.ImageBatch'>
First prediction attributes: ['image', 'gt_label', 'gt_mask', 'mask_path', 'anomaly_map', 'pred_score', 'pred_mask', 'pred_label', 'explanation', 'image_path']
Processing prediction batch 1/7
  Object attributes: ['anomaly_map', 'batch_size', 'collate', 'explanation', 'gt_label', 'gt_mask', 'image', 'image_path', 'item_class', 'items', 'mask_path', 'numpy_class', 'pred_label', 'pred_mask', 'pred_score', 'to_numpy', 'update', 'validate_anomaly_map', 'validate_explanation', 'validate_gt_label', 'validate_gt_mask', 'validate_image', 'validate_image_path', 'validate_mask_path', 'validate_pred_label', 'validate_pred_mask', 'validate_pred_score']
  ✅ Found 16 predictions in .pred_label
  ✅ Found 16 labels in .gt_label
  Batch totals: 0 scores, 16 preds, 16 labels
Raw extraction

In [ ]:
# VISUALIZATION LOGGING TO WANDB
from sklearn.metrics import roc_curve, auc, precision_recall_curve
import matplotlib.pyplot as plt

def create_and_log_plots_to_wandb(analysis_results, wandb_logger):
    """Create visualizations and log them to wandb."""
    
    if analysis_results is None or len(analysis_results['anomaly_scores']) == 0:
        print("⚠️ No data available for plotting")
        return
    
    print("📊 Creating and logging plots to wandb...")
    
    # Extract data
    ground_truths = analysis_results['ground_truths']
    anomaly_scores = analysis_results['anomaly_scores']
    threshold = analysis_results['threshold']
    
    # Create figure with subplots
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # 1. ROC Curve
    fpr, tpr, roc_thresholds = roc_curve(ground_truths, anomaly_scores)
    roc_auc = auc(fpr, tpr)
    
    axes[0,0].plot(fpr, tpr, color='darkorange', lw=2, 
                   label=f'ROC curve (AUC = {roc_auc:.4f})')
    axes[0,0].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    axes[0,0].set_xlim([0.0, 1.0])
    axes[0,0].set_ylim([0.0, 1.05])
    axes[0,0].set_xlabel('False Positive Rate')
    axes[0,0].set_ylabel('True Positive Rate')
    axes[0,0].set_title('ROC Curve')
    axes[0,0].legend(loc="lower right")
    axes[0,0].grid(True)
    
    # 2. Precision-Recall Curve
    precision, recall, pr_thresholds = precision_recall_curve(ground_truths, anomaly_scores)
    pr_auc = auc(recall, precision)
    
    axes[0,1].plot(recall, precision, color='blue', lw=2,
                   label=f'PR curve (AUC = {pr_auc:.4f})')
    axes[0,1].set_xlabel('Recall')
    axes[0,1].set_ylabel('Precision')
    axes[0,1].set_title('Precision-Recall Curve')
    axes[0,1].legend()
    axes[0,1].grid(True)
    
    # 3. Score Distribution
    normal_scores = anomaly_scores[ground_truths == 0]
    abnormal_scores = anomaly_scores[ground_truths == 1]
    
    axes[1,0].hist(normal_scores, bins=30, alpha=0.7, label='Normal', color='green')
    axes[1,0].hist(abnormal_scores, bins=30, alpha=0.7, label='Abnormal', color='red')
    if threshold is not None:
        axes[1,0].axvline(threshold, color='black', linestyle='--', 
                          label=f'Threshold = {threshold:.4f}')
    axes[1,0].set_xlabel('Anomaly Score')
    axes[1,0].set_ylabel('Frequency')
    axes[1,0].set_title('Score Distribution')
    axes[1,0].legend()
    axes[1,0].grid(True)
    
    # 4. Confusion Matrix Heatmap
    import seaborn as sns
    cm = analysis_results['confusion_matrix']
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['Normal', 'Abnormal'],
                yticklabels=['Normal', 'Abnormal'],
                ax=axes[1,1])
    axes[1,1].set_title('Confusion Matrix')
    axes[1,1].set_ylabel('True Label')
    axes[1,1].set_xlabel('Predicted Label')
    
    plt.tight_layout()
    
    # Log the complete figure to wandb
    wandb_logger.experiment.log({"analysis_plots": wandb.Image(fig)})
    
    # Log individual plot metrics
    plot_metrics = {
        "plots/roc_auc": roc_auc,
        "plots/pr_auc": pr_auc,
        "plots/optimal_threshold_roc": roc_thresholds[np.argmax(tpr - fpr)],
    }
    
    wandb_logger.experiment.log(plot_metrics)
    
    plt.show()
    
    print("✅ Plots logged to wandb!")
    return plot_metrics

# Create and log plots if we have analysis results
if 'analysis_results' in locals() and analysis_results is not None:
    plot_metrics = create_and_log_plots_to_wandb(analysis_results, wandb_logger)
else:
    print("⚠️ No analysis results available for plotting")


In [ ]:
# FINAL WANDB SUMMARY AND CLEANUP
import time

# Calculate total experiment duration
total_duration = time.time() - start_time

# Log final experiment summary
final_summary = {
    "experiment/status": "completed",
    "experiment/total_duration_minutes": total_duration / 60,
    "experiment/total_duration_seconds": total_duration,
    "model/architecture": "PatchCore",
    "model/backbone": "resnet18",
    "dataset/name": "spider_test",
    "dataset/task": "anomaly_detection",
}

# Add final performance summary if available
if 'analysis_results' in locals() and analysis_results is not None:
    final_summary.update({
        "results/final_auroc": analysis_results['metrics']['auroc'],
        "results/final_f1_score": analysis_results['metrics']['f1_score'],
        "results/final_accuracy": analysis_results['metrics']['accuracy'],
        "results/samples_processed": len(analysis_results['ground_truths']),
    })

# Log final summary
wandb_logger.experiment.log(final_summary)

# Log dataset information table
dataset_info = wandb.Table(
    columns=["Metric", "Value"],
    data=[
        ["Dataset Root", "spider_test"],
        ["Normal Samples", "~100"],
        ["Abnormal Samples", "~200"],
        ["Model", "PatchCore"],
        ["Backbone", "resnet18"],
        ["Training Duration", f"{total_duration:.2f} seconds"],
        ["Final AUROC", f"{analysis_results['metrics']['auroc']:.4f}" if 'analysis_results' in locals() and analysis_results is not None else "N/A"],
    ]
)

wandb_logger.experiment.log({"experiment_summary": dataset_info})

print(f"\n🎉 EXPERIMENT COMPLETE!")
print(f"⏱️  Total Duration: {total_duration:.2f} seconds ({total_duration/60:.2f} minutes)")
print(f"🌐 Complete Dashboard: {wandb_logger.experiment.url}")

if 'analysis_results' in locals() and analysis_results is not None:
    print(f"📊 Final AUROC: {analysis_results['metrics']['auroc']:.4f}")
    print(f"📈 Final F1-Score: {analysis_results['metrics']['f1_score']:.4f}")
    print(f"🎯 Final Accuracy: {analysis_results['metrics']['accuracy']:.4f}")

# Finish the wandb run
wandb.finish()
print("✅ Wandb run finished and logged successfully!")
